# Validación Track A: Daily OHLCV Pipeline (Simple)

**Objetivo**: Validación rápida y directa del pipeline daily OHLCV

**Fecha**: 2025-10-28

**Tests clave**:
1. Cobertura de tickers
2. Schema correcto
3. Integridad OHLC
4. Contenido de ejemplo

In [1]:
import polars as pl
from pathlib import Path
import random

pl.Config.set_tbl_rows(20)
random.seed(42)

print('✅ Imports completados')

✅ Imports completados


## 1. Cobertura Total

In [2]:
daily_ohlcv_root = Path('D:/04_TRADING_SMALLCAPS/processed/daily_ohlcv')
ticker_dirs = [d for d in daily_ohlcv_root.iterdir() if d.is_dir()]

print(f"=== COBERTURA ===")
print(f"Tickers procesados: {len(ticker_dirs):,}")
print(f"Tickers esperados: 8,620")
print(f"Cobertura: {len(ticker_dirs)/8620*100:.2f}%")
print()

with_data = sum(1 for d in ticker_dirs if (d/'daily.parquet').exists() and len(pl.read_parquet(d/'daily.parquet')) > 0)

print(f"✅ Tickers con datos: {with_data:,} ({with_data/len(ticker_dirs)*100:.2f}%)")

=== COBERTURA ===
Tickers procesados: 8,617
Tickers esperados: 8,620
Cobertura: 99.97%



✅ Tickers con datos: 8,617 (100.00%)


## 2. Verificación de Schema

In [3]:
sample_tickers = random.sample([d for d in ticker_dirs if (d/'daily.parquet').exists()], min(20, len(ticker_dirs)))

print(f"=== SCHEMA (sample {len(sample_tickers)} tickers) ===")
print()

expected_schema = {
    'ticker': pl.Utf8,
    'date': pl.Date,
    'o': pl.Float64,
    'h': pl.Float64,
    'l': pl.Float64,
    'c': pl.Float64,
    'v': pl.Float64,
    'n': pl.Int64,
    'dollar': pl.Float64
}

schema_ok = 0
for ticker_dir in sample_tickers:
    df = pl.read_parquet(ticker_dir / 'daily.parquet')
    if df.schema == expected_schema:
        schema_ok += 1
    else:
        print(f"❌ {ticker_dir.name}: Schema incorrecto")
        print(f"   Esperado: {expected_schema}")
        print(f"   Obtenido: {df.schema}")

print(f"✅ Schema correcto: {schema_ok}/{len(sample_tickers)}")

=== SCHEMA (sample 20 tickers) ===



✅ Schema correcto: 20/20


## 3. Integridad OHLC

In [4]:
print("=== INTEGRIDAD OHLC ===")
print("Verificando: high >= max(o,c), low <= min(o,c), high >= low")
print()

violations = 0
total_rows = 0

for ticker_dir in sample_tickers:
    df = pl.read_parquet(ticker_dir / 'daily.parquet')
    total_rows += len(df)
    
    v1 = df.filter(pl.col('h') < pl.max_horizontal('o', 'c'))
    v2 = df.filter(pl.col('l') > pl.min_horizontal('o', 'c'))
    v3 = df.filter(pl.col('h') < pl.col('l'))
    
    violations += len(v1) + len(v2) + len(v3)

print(f"Total filas verificadas: {total_rows:,}")
print(f"Violaciones: {violations:,}")
print()

if violations == 0:
    print(f"✅ Integridad OHLC: PERFECTA")
else:
    print(f"❌ {violations:,} violaciones ({violations/total_rows*100:.4f}%)")

=== INTEGRIDAD OHLC ===
Verificando: high >= max(o,c), low <= min(o,c), high >= low



Total filas verificadas: 38,196
Violaciones: 0

✅ Integridad OHLC: PERFECTA


## 4. Contenido de Ejemplo (3 tickers)

In [5]:
print("=== CONTENIDO DE EJEMPLO ===")
print()

for ticker_dir in sample_tickers[:3]:
    df = pl.read_parquet(ticker_dir / 'daily.parquet')
    
    print(f"Ticker: {ticker_dir.name}")
    print(f"  Días: {len(df):,}")
    print(f"  Rango: {df['date'].min()} → {df['date'].max()}")
    print(f"  Precio min/max: ${df['c'].min():.2f} / ${df['c'].max():.2f}")
    print(f"  Volumen promedio: {df['v'].mean():,.0f}")
    print()
    print("  Primeras 3 filas:")
    print(df.head(3))
    print()
    print("-" * 80)
    print()

=== CONTENIDO DE EJEMPLO ===

Ticker: SOCA
  Días: 31
  Rango: 2025-09-05 → 2025-10-21
  Precio min/max: $9.90 / $10.09
  Volumen promedio: 136,327

  Primeras 3 filas:
shape: (3, 9)
┌────────┬────────────┬──────┬──────┬───┬───────┬──────────┬─────┬──────────┐
│ ticker ┆ date       ┆ o    ┆ h    ┆ … ┆ c     ┆ v        ┆ n   ┆ dollar   │
│ ---    ┆ ---        ┆ ---  ┆ ---  ┆   ┆ ---   ┆ ---      ┆ --- ┆ ---      │
│ str    ┆ date       ┆ f64  ┆ f64  ┆   ┆ f64   ┆ f64      ┆ i64 ┆ f64      │
╞════════╪════════════╪══════╪══════╪═══╪═══════╪══════════╪═════╪══════════╡
│ SOCA   ┆ 2025-09-05 ┆ 9.96 ┆ 10.0 ┆ … ┆ 9.9   ┆ 501513.0 ┆ 208 ┆ 5.0034e6 │
│ SOCA   ┆ 2025-09-08 ┆ 9.96 ┆ 9.96 ┆ … ┆ 9.945 ┆ 1300.0   ┆ 3   ┆ 12943.5  │
│ SOCA   ┆ 2025-09-09 ┆ 9.94 ┆ 9.96 ┆ … ┆ 9.96  ┆ 850000.0 ┆ 5   ┆ 8.454e6  │
└────────┴────────────┴──────┴──────┴───┴───────┴──────────┴─────┴──────────┘

--------------------------------------------------------------------------------

Ticker: NKTX
  Días: 1,344
  Ran

## 5. Resumen Final

In [6]:
print("=" * 60)
print("=== VALIDACIÓN TRACK A - DAILY OHLCV ===")
print("=" * 60)
print()
print("✅ Tests completados:")
print("  1. Cobertura de tickers (8,617/8,620)")
print("  2. Schema correcto (9 columnas)")
print("  3. Integridad OHLC (0 violaciones)")
print("  4. Contenido de ejemplo verificado")
print()
print("=" * 60)
print("✅ PIPELINE DAILY OHLCV: LISTO")
print("=" * 60)
print()
print("Siguiente paso:")
print("  → Ejecutar event detectors E1, E4, E7, E8")
print("  → Crear multi-event fuser")

=== VALIDACIÓN TRACK A - DAILY OHLCV ===

✅ Tests completados:
  1. Cobertura de tickers (8,617/8,620)
  2. Schema correcto (9 columnas)
  3. Integridad OHLC (0 violaciones)
  4. Contenido de ejemplo verificado

✅ PIPELINE DAILY OHLCV: LISTO

Siguiente paso:
  → Ejecutar event detectors E1, E4, E7, E8
  → Crear multi-event fuser
